In [58]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import bs4
import lxml
import re

In [41]:
def get_sitemats_list(url: str='https://goldapple.ru/sitemap.xml') -> list[str]:
    try:
        res = requests.get(url, timeout=30).text
    except requests.exceptions.ConnectTimeout as err:
        raise requests.exceptions.ConnectTimeout(f'connection timeout, {err}')
    except requests.exceptions.ConnectionError:
        raise requests.exceptions.ConnectionError(f'check url: {url}')
    except requests.exceptions.HTTPError as err:
        raise requests.exceptions.HTTPError(err)

    try:
        soup = bs4.BeautifulSoup(res, 'xml')
        res = [x.text for x in soup.find_all('loc')]
    except AttributeError as err:
        raise AttributeError(f'could not find attribute "loc", {err}')
    except Exception as err:
        raise f'Error parsing xml data: {err}'

    return res

In [78]:
def get_product_urls(sitemaps: list[str]) -> list[str]:
    res = []
    pattern = re.compile(r'\d{10,12}')
    for sitemap in sitemaps:
        xml_products = requests.get(sitemap).text
        soup_products = bs4.BeautifulSoup(xml_products, 'xml')
        product_urls = [x.text for x in soup_products.find_all('loc')]
        for index, x in enumerate(product_urls):
            product = re.findall(pattern, x)
            if product:
                res.append(product_urls[index])
    return res

In [79]:
sitemaps = get_sitemats_list()
product_urls = get_product_urls(sitemaps)

In [80]:
len(product_urls)

71803